In [1]:
pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd

def get_recommendation_report(ticker_symbol):
    # Get company info
    ticker = yf.Ticker(ticker_symbol)
    company_info = ticker.info

    # Get current market data
    market_data = ticker.history(period='1y')

    # Get financial ratios
    financials = ticker.financials
    balance_sheet = ticker.balance_sheet
    cash_flow = ticker.cashflow
    major_holders = ticker.major_holders

    # Fundamental analysis
    fundamental_analysis = pd.DataFrame({
        'Metric': ['P/E Ratio', 'P/B Ratio', 'Dividend Yield', 'ROE', 'ROA'],
        'Value': [
            company_info.get('trailingPE', 'N/A'),
            company_info.get('priceToBook', 'N/A'),
            company_info.get('dividendYield', 'N/A'),
            company_info.get('returnOnEquity', 'N/A'),
            company_info.get('returnOnAssets', 'N/A')
        ]
    })

    # Technical analysis
    market_data['50DMA'] = market_data['Close'].rolling(window=50).mean()
    market_data['200DMA'] = market_data['Close'].rolling(window=200).mean()

    # Simple RSI calculation (this is an approximation)
    delta = market_data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))

    # Bollinger Bands
    market_data['20SMA'] = market_data['Close'].rolling(window=20).mean()
    market_data['20STD'] = market_data['Close'].rolling(window=20).std()
    market_data['UpperBand'] = market_data['20SMA'] + (market_data['20STD'] * 2)
    market_data['LowerBand'] = market_data['20SMA'] - (market_data['20STD'] * 2)

    technical_analysis = pd.DataFrame({
        'Metric': ['50-Day Moving Average', '200-Day Moving Average', 'RSI', 'Bollinger Upper Band'],
        'Value': [
            market_data['50DMA'].iloc[-1],
            market_data['200DMA'].iloc[-1],
            rsi.iloc[-1],
            market_data['UpperBand'].iloc[-1]
        ]
    })

    # Create report
    report = f'''
Company Brief:
{company_info.get('longBusinessSummary', 'N/A')}

Market Data (Last Close):
{market_data.tail(1)}

Financial Ratios:
{financials}

Debt and Cash Flows:
{balance_sheet}
{cash_flow}

Major Holders:
{major_holders}

Fundamental Analysis:
{fundamental_analysis}

Technical Analysis:
{technical_analysis}
'''

    return report

# Test the function
print(get_recommendation_report('AAPL'))



Company Brief:
Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and magazine service; Apple TV+, which offers exclusive original content; Apple Card, a co-branded credit ca